In [1]:
# importing required libraries
import networkx as nx
import urllib3
http = urllib3.PoolManager()
homer = http.request('GET', 'http://ftp.cs.stanford.edu/pub/sgb/homer.dat')

In [2]:
# function to read in the nodes of a graph from passed input file and returns list of nodes
def read_nodes(gfile):
    # initializing empty nodes array
    nodes = []
    # splitting data into lines
    lines = homer.data.splitlines()
    # iterating over lines and extracting nodes
    for line in lines:
        if len(line) != 0:
            if line[:1] != b'*':
                if b':' not in line[:3]:
                    yield str(line[:2],'utf-8')
                else:
                    break

In [3]:
# reads edges from the input file and returns list of edge pairs
def read_edges(gfile):
    # splitting data into lines
    lines = homer.data.splitlines()
    # iterating over lines and extracting pairwise nodes as edges
    for line in lines:
        if len(line) > 1:
            if b':' in line:
                index = line.index(b':')
                line = line[(index+1):]
                all_rel_nodes = line.split(b';')
                for rel_nodes in all_rel_nodes:
                    rel_nodes = rel_nodes.split(b',')
                    for index_node1 in range(len(rel_nodes)-1):
                        for index_node2 in range(index_node1+1, len(rel_nodes)):
                            pair_of_nodes = (str(rel_nodes[index_node1],'utf-8'), str(rel_nodes[index_node2],'utf-8'))
                            yield pair_of_nodes

In [4]:
# initializing graph object
G = nx.Graph()
G.add_nodes_from(read_nodes(homer))
G.add_edges_from(read_edges(homer))

In [5]:
# function which performs DFS on the graph and returns list of nodes in the order of first visited
def Search(graph, root):
    result = [root]
    def Find(graph, root):
        for new in sorted(graph.neighbors(root)):
            if new not in result:
                result.append(new)       
                Find(graph, new)
    Find(graph, root)
    return result

In [6]:
ulysses = Search(G,'OD')

In [7]:
# function to compute the connected components of the graph and return the components in the order of their root nodes
def connected_components(graph):
    result = []
    nodes_not_visited = graph.nodes()
    while len(nodes_not_visited) != 0:
        nodes_not_visited=sorted(nodes_not_visited)
        root = nodes_not_visited[0]
        connected_component = Search(graph, root)
        result.append(connected_component)
        nodes_not_visited = [k for k in nodes_not_visited if k not in connected_component]
    return result

In [8]:
# storing the connected components of the graph in a variable
character_interactions = connected_components(G)

In [9]:
# checking for correctness of the algorithm
component_sizes = [len(c) for c in character_interactions]
print ("There are 12 connected components in the Iliad: {}" .format(len(component_sizes) == 12))
print ("The giant component has size 542: {}".format(max(component_sizes) == 542))
print ("There are 5 isolated characters: {}".format(len([c for c in component_sizes if c == 1]) == 5))

There are 12 connected components in the Iliad: True
The giant component has size 542: True
There are 5 isolated characters: True
